In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("all_stock_data.csv")
df = pd.DataFrame(data)

In [ ]:
import json
from FiinQuantX import FiinSession, RealTimeData

username = 'DSTC_37@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(
    username=username,
    password=password,
).login()

tickers = df["ticker"].unique().tolist()
valid_data = {}
invalid_tickers = []

for t in tickers:
    try:
        fi_dict = client.FundamentalAnalysis().get_ratios(
            tickers=[t],
            TimeFilter="Quarterly",
            LatestYear=2025,
            NumberOfPeriod=12,
            Consolidated=True
        )

        if not fi_dict or len(fi_dict) == 0:
            invalid_tickers.append(t)
        else:
            # lọc lại chỉ giữ năm 2023-2025
            filtered_records = [
                item for item in fi_dict
                if 2023 <= item.get("year", 0) <= 2025
            ]

            if len(filtered_records) == 0:
                invalid_tickers.append(t)
            else:
                valid_data[t] = filtered_records

    except Exception as e:
        print(f"Lỗi với {t}: {e}")
        invalid_tickers.append(t)

print("✅ Số ticker hợp lệ:", len(valid_data))
print("❌ Số ticker không có dữ liệu:", len(invalid_tickers))
print("Danh sách ticker không có dữ liệu:", invalid_tickers[:20])  # in thử 20 cái đầu

Lỗi với PTM: 'PTM'
✅ Số ticker hợp lệ: 1198
❌ Số ticker không có dữ liệu: 407
Danh sách ticker không có dữ liệu: ['E1VFVN30', 'FUESSV50', 'FUESSVFL', 'FUCTVGF3', 'FUCTVGF4', 'FUCTVGF5', 'FUEABVND', 'FUEBFVND', 'FUEDCMID', 'FUEFCV50', 'FUEIP100', 'FUEKIV30', 'FUEKIVFS', 'FUEKIVND', 'FUEMAVND', 'FUETCC50', 'FUEMAV30', 'FUESSV30', 'FUEVFVND', 'FUEVN100']


In [ ]:
# === LƯU FILE JSON ===
with open("valid_data.json", "w", encoding="utf-8") as f:
    json.dump(valid_data, f, ensure_ascii=False, indent=4)

with open("invalid_tickers.json", "w", encoding="utf-8") as f:
    json.dump(invalid_tickers, f, ensure_ascii=False, indent=4)

print("📂 Đã lưu valid_data.json và invalid_tickers.json")

📂 Đã lưu valid_data.json và invalid_tickers.json


In [ ]:
import json

# Đường dẫn đến file JSON bạn đã lưu
file_path = "/content/drive/MyDrive/DSCT-FTU"

# Đọc file JSON
with open(file_path + "/valid_data.json", "r", encoding="utf-8") as f:
    fi_data = json.load(f)

with open(file_path + "/invalid_tickers.json", "r", encoding="utf-8") as f:
    invalid_tickers = json.load(f)

print("✅ Đã load dữ liệu vào fi_data")
print("Số ticker trong fi_dict:", len(fi_data))
# In thử 1 ticker đầu
first_key = list(fi_data.keys())[0]
print("Ví dụ dữ liệu ticker:", first_key)
print(json.dumps(fi_data[first_key][:2], indent=2))  # in thử 2 phần tử đầu

✅ Đã load dữ liệu vào fi_data
Số ticker trong fi_dict: 1198
Ví dụ dữ liệu ticker: AAM
[
  {
    "organizationId": 2736,
    "ticker": "AAM",
    "year": 2023,
    "quarter": 1,
    "ratios": {
      "ProfitabilityRatio": {
        "EBITMargin": -0.04143011,
        "ROA": 0.072987,
        "ROE": 0.077014,
        "ROIC": 0.03779219
      },
      "ValuationRatios": {
        "BasicEPS": 99.88880655,
        "PriceToBook": 0.52849904,
        "PriceToEarning": 6.9376988
      },
      "Growth": {
        "NetRevenueGrowthYoY": -0.45596839,
        "GrossProfitGrowthYoY": -0.60069024
      }
    }
  },
  {
    "organizationId": 2736,
    "ticker": "AAM",
    "year": 2023,
    "quarter": 2,
    "ratios": {
      "ProfitabilityRatio": {
        "EBITMargin": -0.05010263,
        "ROA": 0.036955,
        "ROE": 0.038859,
        "ROIC": -0.00740753
      },
      "ValuationRatios": {
        "BasicEPS": 69.97369851,
        "PriceToBook": 0.52991354,
        "PriceToEarning": 13.37999074
 

In [ ]:
!pip install --extra-index-url https://fiinquant.github.io/fiinquantx/simple fiinquantx

Looking in indexes: https://pypi.org/simple, https://fiinquant.github.io/fiinquantx/simple


In [ ]:
fi = client.FiinIndicator()


# Tính RSI (mặc định window = 14)
df['rsi'] = fi.rsi(df['close'], window=14)

# Tính MFI (Money Flow Index)
df['mfi'] = fi.mfi(df['high'], df['low'], df['close'], df['volume'], window=14)

# Tính MACD
df['macd'] = fi.macd(df['close'], window_fast=12, window_slow=26)
df['macd_signal'] = fi.macd_signal(df['close'], window_fast=12, window_slow=26, window_sign=9)
df['macd_diff'] = fi.macd_diff(df['close'], window_fast=12, window_slow=26, window_sign=9)

# Tính Bollinger Bands (mặc định window=20, std=2)
df['bollinger_hband'] = fi.bollinger_hband(df['close'], window=20, window_dev=2)
df['bollinger_lband'] = fi.bollinger_lband(df['close'], window=20, window_dev=2)

print(df)


        ticker         timestamp      open      high       low     close  \
0          AAM  2023-01-03 00:00  11284.80  11284.80  11002.68  11190.76   
1          AAM  2023-01-04 00:00  10673.54  11002.68  10673.54  10908.64   
2          AAM  2023-01-05 00:00  10438.44  11190.76  10438.44  11190.76   
3          AAM  2023-01-06 00:00  11002.68  11143.74  10955.66  10955.66   
4          AAM  2023-01-09 00:00  11284.80  11284.80  10908.64  10955.66   
...        ...               ...       ...       ...       ...       ...   
1040146    VXT  2025-08-25 00:00  11000.00  11000.00  11000.00  11000.00   
1040147    VXT  2025-08-26 00:00  11000.00  11000.00  11000.00  11000.00   
1040148    VXT  2025-08-27 00:00  11000.00  11000.00  11000.00  11000.00   
1040149    VXT  2025-08-28 00:00  11000.00  11000.00  11000.00  11000.00   
1040150    VXT  2025-08-29 00:00  11000.00  11000.00  11000.00  11000.00   

         volume      bu      sd         fn         fs         fb        rsi  \
0       

In [ ]:
tickers = df['ticker'].unique().tolist()
tickers[:10]

['AAM', 'AAT', 'ABR', 'ABS', 'ABT', 'ACB', 'ACC', 'ACG', 'ACL', 'ADP']

In [ ]:
# Cell 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

def merge_fundamental_data(df, fi_data, invalid_tickers):
    """
    Ghép dữ liệu fundamental từ fi_data vào DataFrame chứng khoán.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame chứa dữ liệu giá chứng khoán với cột timestamp
    fi_data : list
    invalid_tickers : list
        List các ticker không có fundamental data

    Returns:
    --------
    pandas.DataFrame
        DataFrame đã được enriched với các cột fundamental
    """

    # Tạo copy của DataFrame gốc
    df_enriched = df.copy()

    # Chuyển đổi timestamp thành datetime nếu chưa phải
    if not pd.api.types.is_datetime64_any_dtype(df_enriched['timestamp']):
        df_enriched['timestamp'] = pd.to_datetime(df_enriched['timestamp'])

    # Thêm cột year và quarter từ timestamp
    df_enriched['year'] = df_enriched['timestamp'].dt.year
    df_enriched['quarter'] = df_enriched['timestamp'].dt.quarter

    # Khởi tạo các cột fundamental với NaN
    fundamental_columns = ['PB', 'PE', 'ROE', 'EPS', 'RevenueGrowth', 'ProfitGrowth']
    for col in fundamental_columns:
        df_enriched[col] = np.nan

    # Tạo dictionary mapping từ fi_data để tìm kiếm nhanh hơn
    fundamental_lookup = {}

    # Xử lý cả trường hợp fi_data là list hoặc dict
    processed_count = 0
    skipped_count = 0

    if isinstance(fi_data, list):
        # Trường hợp fi_data là list
        for record in fi_data:
            # Kiểm tra các trường bắt buộc
            if not all(key in record for key in ['ticker', 'year', 'quarter']):
                skipped_count += 1
                continue

            ticker = record['ticker']
            if ticker in invalid_tickers:  # Bỏ qua ticker không hợp lệ
                skipped_count += 1
                continue

            # Kiểm tra có trường ratios không
            if 'ratios' not in record or record['ratios'] is None:
                skipped_count += 1
                continue

            try:
                key = (ticker, record['year'], record['quarter'])
                ratios = record['ratios']
                fundamental_lookup[key] = {
                    'PB': ratios.get('ValuationRatios', {}).get('PriceToBook'),
                    'PE': ratios.get('ValuationRatios', {}).get('PriceToEarning'),
                    'ROE': ratios.get('ProfitabilityRatio', {}).get('ROE'),
                    'EPS': ratios.get('ValuationRatios', {}).get('BasicEPS'),
                    'RevenueGrowth': ratios.get('Growth', {}).get('NetRevenueGrowthYoY'),
                    'ProfitGrowth': ratios.get('Growth', {}).get('GrossProfitGrowthYoY')
                }
                processed_count += 1
            except (KeyError, AttributeError, TypeError) as e:
                skipped_count += 1
                continue
    else:
        raise ValueError("fi_data must be either a dictionary or a list")

    print(f"Đã xử lý {processed_count} records fundamental, bỏ qua {skipped_count} records thiếu dữ liệu")

    # Áp dụng fundamental data vào DataFrame
    def apply_fundamental(row):
        ticker = row['ticker']

        # Nếu ticker nằm trong invalid_tickers, giữ nguyên NaN
        if ticker in invalid_tickers:
            return row

        # Tạo key để lookup
        key = (ticker, row['year'], row['quarter'])

        # Tìm và áp dụng fundamental data
        if key in fundamental_lookup:
            fund_data = fundamental_lookup[key]
            for col in fundamental_columns:
                if fund_data[col] is not None:
                    row[col] = fund_data[col]

        return row

    # Áp dụng function cho từng row
    print("Đang ghép dữ liệu fundamental...")
    df_enriched = df_enriched.apply(apply_fundamental, axis=1)

    # Xóa các cột trung gian
    df_enriched = df_enriched.drop(['year', 'quarter'], axis=1)

    # Thống kê kết quả
    total_rows = len(df_enriched)
    rows_with_fundamental = df_enriched[fundamental_columns].notna().any(axis=1).sum()

    print(f"Hoàn thành! Đã ghép fundamental cho {rows_with_fundamental}/{total_rows} dòng dữ liệu")
    print(f"Đã xử lý {processed_count} records fundamental từ fi_data")
    print(f"Các cột fundamental đã thêm: {', '.join(fundamental_columns)}")

    return df_enriched


df_enriched = merge_fundamental_data(df, fi_data, invalid_tickers)

Đã xử lý 11022 records fundamental, bỏ qua 321 records thiếu dữ liệu
Đang ghép dữ liệu fundamental...
Hoàn thành! Đã ghép fundamental cho 673248/1040151 dòng dữ liệu
Đã xử lý 11022 records fundamental từ fi_data
Các cột fundamental đã thêm: PB, PE, ROE, EPS, RevenueGrowth, ProfitGrowth
=== THỐNG KÊ DỮ LIỆU FUNDAMENTAL ===
Tổng số dòng dữ liệu: 1040151
PB: 663312 dòng có dữ liệu (63.8%)
PE: 662334 dòng có dữ liệu (63.7%)
ROE: 663378 dòng có dữ liệu (63.8%)
EPS: 669557 dòng có dữ liệu (64.4%)
RevenueGrowth: 655426 dòng có dữ liệu (63.0%)
ProfitGrowth: 655737 dòng có dữ liệu (63.0%)

Số ticker có dữ liệu fundamental: 1153/1605
  ticker  rows_with_fundamental
0    A32                      0
1    AAA                    618
2    AAH                    362
3    AAM                    618
4    AAS                    618
  ticker  timestamp     close        PB        PE       ROE
0    AAM 2023-01-03  11190.76  0.528499  6.937699  0.077014
1    AAM 2023-01-04  10908.64  0.528499  6.937699  0.077

In [ ]:
df_enriched.loc[df_enriched["ticker"] == 'AAT']

,ticker,timestamp,open,high,low,close,volume,bu,sd,fn,...,macd_signal,macd_diff,bollinger_hband,bollinger_lband,PB,PE,ROE,EPS,RevenueGrowth,ProfitGrowth
662,AAT,2023-01-03,4468.464,4594.590,4459.455,4594.590,351439.0,185700.0,127300.0,74941000.0,...,-35.554093,-189.398059,8363.472214,5970.986786,0.584335,4.661105,0.123658,22.965874,0.099728,0.513251
663,AAT,2023-01-04,4630.626,4630.626,4558.554,4603.599,242433.0,109300.0,125700.0,94516000.0,...,-105.722240,-280.672588,8670.156320,5399.662580,0.584335,4.661105,0.123658,22.965874,0.099728,0.513251
664,AAT,2023-01-05,4594.590,4621.617,4549.545,4558.554,179984.0,59700.0,104000.0,41937000.0,...,-186.992037,-325.079191,8850.089625,4945.584675,0.584335,4.661105,0.123658,22.965874,0.099728,0.513251
665,AAT,2023-01-06,4594.590,4594.590,4486.482,4495.491,294723.0,83600.0,199800.0,-41280000.0,...,-271.539778,-338.190964,8956.867982,4554.355418,0.584335,4.661105,0.123658,22.965874,0.099728,0.513251
666,AAT,2023-01-09,4504.500,4639.635,4369.365,4639.635,527134.0,90100.0,238400.0,11365000.0,...,-350.791311,-317.006130,8971.808638,4256.378262,0.584335,4.661105,0.123658,22.965874,0.099728,0.513251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,AAT,2025-08-25,3440.000,3440.000,3320.000,3370.000,82652.0,6100.0,74000.0,0.0,...,40.561967,-21.867071,3566.676528,3341.323472,NaN,NaN,NaN,NaN,NaN,NaN
1320,AAT,2025-08-26,3360.000,3400.000,3350.000,3400.000,22773.0,4500.0,17200.0,0.0,...,35.551272,-20.042783,3564.399036,3346.600964,NaN,NaN,NaN,NaN,NaN,NaN
1321,AAT,2025-08-27,3400.000,3450.000,3360.000,3450.000,59290.0,10800.0,22000.0,0.0,...,31.805794,-14.981910,3564.701426,3347.298574,NaN,NaN,NaN,NaN,NaN,NaN
1322,AAT,2025-08-28,3450.000,3470.000,3410.000,3460.000,92620.0,29300.0,61900.0,13791000.0,...,29.136727,-10.676267,3553.705457,3350.294543,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_enriched.to_csv("full_data.csv", index=False, encoding="utf-8-sig")

In [ ]:
df_enriched.shape

(1040151, 25)

In [ ]:
df.shape

(1040151, 19)